# Finetuning FastPitch for a new speaker

In this tutorial, we will finetune a single speaker FastPitch (with alignment) model on limited amount of new speaker's data. We cover two finetuning techniques: 
1. We finetune the model parameters only on new speaker's text and speech pairs; 
2. We add a learnable speaker embedding layer to the model, and finetune on a mixture of original speaker's and new speaker's data.

We will first prepare filelists containing the audiopaths and text of the samples on which we wish to finetune the model, then generate and run a training command to finetune Fastpitch on 5 mins of data, and finally synthesize the audio from the trained checkpoint.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
BRANCH = 'r1.6.1'
# # If you're using Google Colab and not running locally, uncomment and run this cell.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode
# !python -m pip install git+https://github.com/NeMo/NeMo.git@$BRANCH#egg=nemo_toolkit[tts]

We will first create filelists of audio on which we wish to finetune the FastPitch model. We will create two kinds of filelists, one which contains only the audio files of the new speaker and one which contains the mixed audio files of the new speaker and the speaker used for training the pre-trained FastPitch Checkpoint.

<div class="alert alert-block alert-warning">
    WARNING: This notebook requires downloading the HiFiTTS dataset which is 41GB. We plan on reducing the amount the download amount.
</div>

In [ ]:
!wget https://nemo-public.s3.us-east-2.amazonaws.com/6097_5_mins.tar.gz  # Contains 10MB of data
!tar -xzf 6097_5_mins.tar.gz

Looking at manifest.json, we see a standard NeMo json that contains the filepath, text, and duration. Please note that manifest.json only contains the relative path.


def make_sub_file_list(speaker_id, clean_other, split, num_samples, total_duration_mins, seed=42):
    """
    Creates a subset of training data for a HiFiTTS speaker. Specify either the num_samples or total_duration_mins
    Saves the filelist in the filelist_dir. split is either "train" or "dev"
    
    Arguments:
    speaker_id -- speaker id of the new HiFiTTS speaker
    clean_other -- "clean" or "other" depending on type of data of new HiFiTTS speaker
    split -- "train" or "dev"
    num_samples -- Number samples of new speaker (set None if specifying total_duration_mins)
    total_duration_mins -- Total duration of new speaker's data (set None if specifying num_samples)
    """
    file_list_name = "{}_manifest_{}_{}.json".format(speaker_id, clean_other, split)
    with open(os.path.join(data_dir, file_list_name), 'r') as f:
        all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    random.seed(seed)
    random.shuffle(all_records)
    
    if num_samples is not None and total_duration_mins is None:
        sub_records = all_records[:num_samples]
        fname_extension = "ns_{}".format(num_samples)
    elif num_samples is None and total_duration_mins is not None:
        sub_record_duration = 0.0
        sub_records = []
        for r in all_records:
            sub_record_duration += r['duration']
            if sub_record_duration > total_duration_mins*60.0:
                print ("Duration reached {} mins using {} records".format(total_duration_mins, len(sub_records)))
                break
            sub_records.append(r)
        fname_extension = "dur_{}_mins".format( int(round(total_duration_mins )))
    elif num_samples is None and total_duration_mins is None:
        sub_records = all_records
        fname_extension = "ns_all"
    else:
        raise NotImplementedError()
    print ("num sub records", len(sub_records))
    
    if not os.path.exists(filelist_dir):
        os.makedirs(filelist_dir)
    
    target_fp = os.path.join(filelist_dir, "{}_mainifest_{}_{}_local.json".format(speaker_id, split,  fname_extension))
    with open(target_fp, 'w') as f:
        for record in json.loads(json.dumps(sub_records)):
            record['audio_filepath'] = record['audio_filepath'][record['audio_filepath'].find("wav/"):]
            record['audio_filepath'] = os.path.join(data_dir, record['audio_filepath']) 
            f.write(json.dumps(record) + "\n")

def mix_file_list(speaker_id, clean_other, split, num_samples, total_duration_mins, original_speaker_id, original_clean_other, n_orig=None, seed=42):
    """
    Creates a mixed dataset of new and original speaker. num_samples or total_duration_mins specifies the amount 
    of new speaker data to be used and n_orig specifies the number of original speaker samples. Creates a balanced 
    dataset with alternating new and old speaker samples. Saves the filelist in the filelist_dir. 
    
    Arguments:
    speaker_id -- speaker id of the new HiFiTTS speaker
    clean_other -- "clean" or "other" depending on type of data of new HiFiTTS speaker
    split -- "train" or "dev"
    num_samples -- Number samples of new speaker (set None if specifying total_duration_mins)
    total_duration_mins -- Total duration of new speaker's data (set None if specifying num_samples)
    original_speaker_id -- speaker id of the original HiFiTTS speaker (on which FastPitch was trained)
    original_clean_other -- "clean" or "other" depending on type of data of new HiFiTTS speaker
    n_orig -- Number of samples of old speaker to be mixed with new speaker
    
    """
    file_list_name = "{}_manifest_{}_{}.json".format(speaker_id, clean_other, split)
    with open(os.path.join(data_dir, file_list_name), 'r') as f:
        all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    
    original_file_list_name = "{}_manifest_{}_{}.json".format(original_speaker_id, original_clean_other, "train")
    with open(os.path.join(data_dir, original_file_list_name), 'r') as f:
        original_all_records = [json.loads(l) for l in f.read().split("\n") if len(l) > 0]
    for r in original_all_records:
        r['audio_filepath'] = r['audio_filepath'][r['audio_filepath'].find("wav/"):]
    
    random.seed(seed)
    if n_orig is not None:
        random.shuffle(original_all_records)
        original_all_records = original_all_records[:n_orig]
        
    random.seed(seed)
    random.shuffle(all_records)
    
    if num_samples is not None and total_duration_mins is None:
        sub_records = all_records[:num_samples]
        fname_extension = "ns_{}".format(num_samples)
    elif num_samples is None and total_duration_mins is not None:
        sub_record_duration = 0.0
        sub_records = []
        for r in all_records:
            sub_record_duration += r['duration']
            if sub_record_duration > total_duration_mins * 60.0:
                print ("Duration reached {} mins using {} records".format(total_duration_mins, len(sub_records)))
                break
            sub_records.append(r)
        fname_extension = "dur_{}_mins".format( int(round(total_duration_mins)))
    elif num_samples is None and total_duration_mins is None:
        sub_records = all_records
        fname_extension = "ns_all"
    else:
        raise NotImplementedError()
        
    print(len(original_all_records))
    
    if not os.path.exists(filelist_dir):
        os.makedirs(filelist_dir)
        
    target_fp = os.path.join(filelist_dir, "{}_mainifest_{}_{}_local_mix_{}.json".format(speaker_id, split,  fname_extension, original_speaker_id))
    with open(target_fp, 'w') as f:
        for ridx, original_record in enumerate(original_all_records):
            original_record['audio_filepath'] = original_record['audio_filepath'][original_record['audio_filepath'].find("wav/"):]
            original_record['audio_filepath'] = os.path.join(data_dir, original_record['audio_filepath']) 
            
            new_speaker_record = sub_records[ridx % len(sub_records)]
            new_speaker_record['audio_filepath'] = new_speaker_record['audio_filepath'][new_speaker_record['audio_filepath'].find("wav/"):]
            new_speaker_record['audio_filepath'] = os.path.join(data_dir, new_speaker_record['audio_filepath']) 
            
            new_speaker_record['speaker'] = 1
            original_record['speaker'] = 0
            f.write(json.dumps(original_record) + "\n")
            f.write(json.dumps(new_speaker_record) + "\n")

In [ ]:
make_sub_file_list(92, "clean", "train", None, 5)
mix_file_list(92, "clean", "train", None, 5, 8051, "other", n_orig=5000)
make_sub_file_list(92, "clean", "dev", None, None)

## Finetuning the model on filelists

Let's first download the pretrained checkpoint that we want to finetune from. NeMo will save checkpoints to ~/.cache, so let's move that to our current directory. 

*Note: please, check that `home_path` refers to your home folder. Otherwise, change it manually.*

In [ ]:
home_path = !(echo $HOME)
home_path = home_path[0] 
print(home_path)

In [ ]:
import os
import json

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

from nemo.collections.tts.models import FastPitchModel
FastPitchModel.from_pretrained("tts_en_fastpitch")

from pathlib import Path
nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
print(f"Copying {nemo_files[0]} to ./")
Path("./tts_en_fastpitch_align.nemo").write_bytes(nemo_files[0].read_bytes())

To finetune the FastPitch model on the above created filelists, we use `examples/tts/fastpitch_finetune.py` script to train the models with the `fastpitch_align.yaml` configuration.

Let's grab those files.

In [ ]:
!wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/fastpitch_finetune.py
!mkdir -p conf && cd conf && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/fastpitch_align.yaml && cd ..

The generated command should look something like this. We can ofcourse tweak things like epochs/learning rate if we like

**NOTE: This will take about 50 minutes on colab's K80 GPUs.**

`python fastpitch_finetune.py --config-name=fastpitch_align.yaml train_dataset=./6097_manifest_train_dur_5_mins_local.json validation_datasets=./6097_manifest_dev_ns_all_local.json +init_from_nemo_model=./tts_en_fastpitch_align.nemo +trainer.max_steps=1000 ~trainer.max_epochs trainer.check_val_every_n_epoch=25 prior_folder=./Priors6097 model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 exp_manager.exp_dir=./ljspeech_to_6097_no_mixing_5_mins model.n_speakers=1 model.pitch_avg=121.9 model.pitch_std=23.1 model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null`

In [ ]:
!(python fastpitch_finetune.py --config-name=fastpitch_align.yaml \
  train_dataset=./6097_manifest_train_dur_5_mins_local.json \
  validation_datasets=./6097_manifest_dev_ns_all_local.json \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  prior_folder=./Priors6097 \
  model.train_ds.dataloader_params.batch_size=24 \
  model.validation_ds.dataloader_params.batch_size=24 \
  exp_manager.exp_dir=./ljspeech_to_6097_no_mixing_5_mins \
  model.n_speakers=1 model.pitch_avg=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
)

Let's take a closer look at the training command:

* `python fastpitch_finetune.py --config-name=fastpitch_align.yaml`
  * --config-name tells the script what config to use.

* `train_dataset=./6097_manifest_train_dur_5_mins_local.json validation_datasets=./6097_manifest_dev_ns_all_local.json`
  * We tell the model what manifest files we can to train and eval on.

* `+init_from_nemo_model=./tts_en_fastpitch_align.nemo`
  * We tell the script what checkpoint to finetune from.

* `+trainer.max_steps=1000 ~trainer.max_epochs trainer.check_val_every_n_epoch=25`
  * For this experiment, we need to tell the script to train for 1000 training steps/iterations. We need to remove max_epochs using `~trainer.max_epochs`.

* `prior_folder=./Priors6097 model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24`
  * Some dataset parameters. The dataset does some online processing and stores the processing steps to the `prior_folder`.

* `exp_manager.exp_dir=./ljspeech_to_6097_no_mixing_5_mins`
  * Where we want to save our log files, tensorboard file, checkpoints, and more

* `model.n_speakers=1`
  * The number of speakers in the data. There is only 1 for now, but we will revisit this parameter later in the notebook

* `model.pitch_avg=121.9 model.pitch_std=23.1 model.pitch_fmin=30 model.pitch_fmax=512`
  * For the new speaker, we need to define new pitch hyperparameters for better audio quality.
  * These parameters work for speaker 6097 from the HiFiTTS dataset
  * For speaker 92, we suggest `model.pitch_avg=214.5 model.pitch_std=30.9 model.pitch_fmin=80 model.pitch_fmax=512`
  * fmin and fmax are hyperparameters to librosa's pyin function. We recommend tweaking these per speaker.
  * After fmin and fmax are defined, pitch mean and std can be easily extracted

* `model.optim.lr=2e-4 ~model.optim.sched model.optim.name=adam`
  * For fine-tuning, we lower the learning rate
  * We use a fixed learning rate of 2e-4
  * We switch from the lamb optimizer to the adam optimizer

* `trainer.devices=1 trainer.strategy=null`
  * For this notebook, we default to 1 gpu which means that we do not need ddp
  * If you have the compute resources, feel free to scale this up to the number of free gpus you have available
  * Please remove the `trainer.strategy=null` section if you intend on multi-gpu training

## Synthesize samples from finetuned checkpoints

Once we have finetuned our FastPitch model, we can synthesize the audio samples for given text using the following inference steps. We use a HiFiGAN vocoder trained on multiple speakers, get the trained checkpoint path for our trained model and synthesize audio for a given text as follows.

In [ ]:
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

In [ ]:
def infer(spec_gen_model, vocoder_model, str_input, speaker = None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Arguments:
    spec_gen_model -- Instance of FastPitch model
    vocoder_model -- Instance of a vocoder model (HiFiGAN in our case)
    str_input -- Text input for the synthesis
    speaker -- Speaker number (in the case of a multi-speaker model -- in the mixing case)
    
    Returns:
    spectrogram, waveform of the synthesized audio.
    """
    parser_model = spec_gen_model
    with torch.no_grad():
        parsed = parser_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().cuda()
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker = speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt(experiment_base_dir, new_speaker_id, duration_mins, mixing_enabled, original_speaker_id):
    """
    Gives the model checkpoint paths of an experiment  we ran. 
    
    Arguments:
    experiment_base_dir -- Base experiment directory (specified on top of this notebook as exp_base_dir)
    new_speaker_id -- Speaker id of new HiFiTTS speaker we finetuned FastPitch on
    duration_mins -- total minutes of the new speaker data
    mixing_enabled -- True or False depending on whether we want to mix the original speaker data or not
    original_speaker_id -- speaker id of the original HiFiTTS speaker
    
    Returns:
    List of all checkpoint paths sorted by validation error, Last checkpoint path
    """
    if not mixing_enabled:
        exp_dir = "{}/{}_to_{}_no_mixing_{}_mins".format(experiment_base_dir, original_speaker_id, new_speaker_id, duration_mins)
    else:
        exp_dir = "{}/{}_to_{}_mixing_{}_mins".format(experiment_base_dir, original_speaker_id, new_speaker_id, duration_mins)
    
    ckpt_candidates = []
    last_ckpt = None
    for root, dirs, files in os.walk(exp_dir):
        for file in files:
            if file.endswith(".ckpt"):
                val_error = float(file.split("v_loss=")[1].split("-epoch")[0])
                if "last" in file:
                    last_ckpt = os.path.join(root, file)
                ckpt_candidates.append( (val_error, os.path.join(root, file)))
    ckpt_candidates.sort()
    
    return ckpt_candidates, last_ckpt

Specify the speaker id, duration mins and mixing variable to find the relevant checkpoint from the exp_base_dir and compare the synthesized audio with validation samples of the new speaker.

In [ ]:
new_speaker_id = 92
duration_mins = 5
mixing = False
original_speaker_id = 8051


_ ,last_ckpt = get_best_ckpt(exp_base_dir, new_speaker_id, duration_mins, mixing, original_speaker_id)
print(last_ckpt)

spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()
_speaker=None
if mixing:
    _speaker = 1

num_val = 2

manifest_path = os.path.join(filelist_dir, "{}_mainifest_dev_ns_all_local.json".format(new_speaker_id))
val_records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        val_records.append( json.loads(line) )
        if len(val_records) >= num_val:
            break
            
for val_record in val_records:
    print ("Real validation audio")
    ipd.display(ipd.Audio(val_record['audio_filepath'], rate=44100))
    print ("SYNTHESIZED FOR -- Speaker: {} | Dataset size: {} mins | Mixing:{} | Text: {}".format(new_speaker_id, duration_mins, mixing, val_record['text']))
    spec, audio = infer(spec_model, vocoder, val_record['text'], speaker = _speaker)
    ipd.display(ipd.Audio(audio, rate=44100))
    %matplotlib inline
    #if spec is not None:
    imshow(spec, origin="lower", aspect = "auto")
    plt.show()

## Improving Speech Quality
___

We see that from fine-tuning FastPitch, we were able to generate audio in a male voice but the audio quality is not as good as we expect. We recommend two steps to improve audio quality:

* Finetuning HiFiGAN
* Adding more data

Both of these steps are outside the scope of the notebook due to the limited compute available on colab.

### Finetuning HiFiGAN
From the synthesized samples, there might be audible audio crackling. To fix this, we need to finetune HiFiGAN on the new speaker's data. HiFiGAN shows improvement using synthesized mel spectrograms, so the first step is to generate mel spectrograms with our finetuned FastPitch model.

```python
# Get records from the training manifest
manifest_path = "./6097_manifest_train_dur_5_mins_local.json"
records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

# Generate a spectrogram for each item
for i, r in enumerate(records):
  with torch.no_grad():
      parsed = spec_gen_model.parse(r['text'])
      spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=1)
      if isinstance(spectrogram, torch.Tensor):
          spectrogram = spectrogram.to('cpu').numpy()
      if len(spectrogram.shape) == 3:
          spectrogram = spectrogram[0]
      np.save(f"mel_{i}", spectrogram)
      r["mel_filepath"] = f"mel_{i}.npy"

# Save to a new json
with open("hifigan_train_ft.json", "w") as f:
  for r in records:
    f.write(json.dumps(r) + '\n')

# Please do the same for the validation json. Code is omitted.
```

We can then finetune hifigan similarly to fastpitch using NeMo's [hifigan_finetune.py](https://github.com/NVIDIA/NeMo/blob/main/examples/tts/hifigan_finetune.py) and [hifigan.yaml](https://github.com/NVIDIA/NeMo/blob/main/examples/tts/conf/hifigan/hifigan.yaml):

`python examples/tts/hifigan_finetune.py --config_name=hifigan.yaml model.train_ds.dataloader_params.batch_size=32 model.max_steps=1000 ~model.sched model.optim.lr=0.0001 train_dataset=./hifigan_train_ft.json validation_datasets=./hifigan_val_ft.json exp_manager.exp_dir=hifigan_ft +init_from_nemo_model=tts_hifigan.nemo trainer.check_val_every_n_epoch=10 model/train_ds=train_ds_finetune`

### Improving TTS by Adding More Data
We can add more data in two ways. they can be combined for the best effect:

* Add more training data from the new speaker

The entire notebook can be repeated from the top after a new .json is defined for the additional data. Modify your finetuning commands to point to the new json. Be sure to increase the number of steps as more data is added to both the fastpitch and hifigan finetuning. We recommend 1000 steps per minute of audio for fastpitch and 500 steps per minute of audio for hifigan.

* Mix new speaker data with old speaker data

We recommend to train fastpitch using both old speaker data (LJSpeech in this notebook) and the new speaker data. In this case, please modify the .json when finetuning fastpitch to include speaker information:

`
{"audio_filepath": "new_speaker.wav", "text": "sample", "duration": 2.6, "speaker": 1}
{"audio_filepath": "old_speaker.wav", "text": "LJSpeech sample", "duration": 2.6, "speaker": 0}
`
5 hours of data from the old speaker should be sufficient. Since we should have less data from the new speaker, we need to ensure that the model sees a similar amount of new data and old data. For each sample from the old speaker, please add a sample from the new speaker in the .json. The samples from the new speaker will be repeated.

Modify the fastpitch training command to point to the new training and validation .jsons, and update `model.n_speakers=1` to `model.n_speakers=2`. Ensure the pitch statistics correspond to the new speaker.

For HiFiGAN finetuning, the training should be done on the new speaker data.